In [1]:
import os
import sys
import re

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/spark/third-party-jars/spark-streaming-kafka-0-10_2.12-3.1.1.jar;/usr/local/spark/third-party-jars/spark-sql-kafka-0-10_2.12-3.1.1.jar pyspark-shell'

# Configure the environment
# if 'SPARK_HOME' not in os.environ:
# os.environ['SPARK_HOME'] = '/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7'
# Create a variable for our root path
# SPARK_HOME = os.environ['SPARK_HOME']
# print(SPARK_HOME)
#Set python location 
# sys.path.append("/usr/local/lib/python3.7/site-packages")
#Set python location 
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python")
# sys.path.append("/home/zdeploy/spark/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip")


from pyspark.sql import SparkSession, SQLContext
from pyspark import  SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# Spark session & context
conf = SparkConf()
conf.setMaster("local").setAppName("batch-job")
conf.set("spark.executor.memory", "1g")
conf.set("spark.executor.cores", "1")
conf.set("spark.cores.max", "4")
conf.set("spark.driver.memory",'1g')
conf.set("spark.driver.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.executor.extraClassPath", "/usr/local/spark/third-party-jars/*")
conf.set("spark.sql.caseSensitive", "true")
conf.set("spark.ui.port", "4040")

sc = SparkContext(conf=conf)
spark = SparkSession(sc)
sqlContext = SQLContext(sc)
ssc = StreamingContext(sc, 1)

In [24]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
.container { 
    width:95% !important; 
}
</style>

# Process static data

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

from datetime import datetime
from pytz import timezone
import math

In [4]:
#udf

# check if segment is not an empty segment (two vertex is the same)
@udf(returnType=BooleanType())
def is_not_the_same(from_latlon, to_latlon):
    return (from_latlon[0] != to_latlon[0]) | (from_latlon[1] != to_latlon[1])

@udf(returnType=LongType())
def file_path_to_ts(file_path):
    print(file_path)
    return int(file_path[-18:-4])

@udf(returnType=IntegerType())
def get_day_type(epoch_seconds):
    dt = datetime.fromtimestamp(epoch_seconds)
    wd = dt.astimezone(timezone('America/Los_Angeles')).weekday()
    return 0 if wd < 5 else 1

@udf(returnType=DoubleType())
def haversine_meter(lat1, lon1, lat2, lon2):
    # distance between latitudes and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formulae
    a = (math.pow(math.sin(dLat / 2), 2) +
         math.pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c * 1000

In [5]:
routes = spark.read.csv('hdfs://namenode:8020/ola/static_data/routes.txt', header=True)
routes.select("route_id").show(100)

+---------+
| route_id|
+---------+
|  2-13139|
|  4-13139|
| 10-13139|
| 14-13139|
| 16-13139|
| 18-13139|
| 20-13139|
| 28-13139|
| 30-13139|
| 33-13139|
| 35-13139|
| 40-13139|
| 45-13139|
| 51-13139|
| 53-13139|
| 55-13139|
| 60-13139|
| 62-13139|
| 66-13139|
| 68-13139|
| 70-13139|
| 71-13139|
| 76-13139|
| 78-13139|
| 81-13139|
| 83-13139|
| 90-13139|
| 92-13139|
| 94-13139|
| 96-13139|
|102-13139|
|105-13139|
|106-13139|
|108-13139|
|110-13139|
|111-13139|
|115-13139|
|117-13139|
|120-13139|
|125-13139|
|127-13139|
|128-13139|
|130-13139|
|150-13139|
|152-13139|
|154-13139|
|155-13139|
|158-13139|
|161-13139|
|163-13139|
|164-13139|
|165-13139|
|166-13139|
|167-13139|
|169-13139|
|175-13139|
|176-13139|
|180-13139|
|183-13139|
|200-13139|
|201-13139|
|202-13139|
|204-13139|
|205-13139|
|206-13139|
|207-13139|
|209-13139|
|210-13139|
|211-13139|
|212-13139|
|217-13139|
|218-13139|
|222-13139|
|224-13139|
|230-13139|
|232-13139|
|233-13139|
|234-13139|
|236-13139|
|237-13139|
|239

In [6]:
trips = spark.read.csv('hdfs://namenode:8020/ola/static_data/trips.txt', header=True)
trips.show(1000, False)

+---------+-----------------------+--------------------------------+-------------+------------+--------+-------------+--------------+
|route_id |service_id             |trip_id                         |trip_headsign|direction_id|block_id|shape_id     |perm_trip_id  |
+---------+-----------------------+--------------------------------+-------------+------------+--------+-------------+--------------+
|611-13139|DEC20-D02CAR-1_Weekday |52088401-DEC20-D02CAR-1_Weekday |null         |1           |6110300 |6110028_DEC20|10611000280539|
|611-13139|DEC20-D02CAR-1_Weekday |52088402-DEC20-D02CAR-1_Weekday |null         |1           |6110100 |6110029_DEC20|10611000290605|
|611-13139|DEC20-D02CAR-1_Weekday |52088403-DEC20-D02CAR-1_Weekday |null         |1           |6110200 |6110029_DEC20|10611000290705|
|611-13139|DEC20-D02CAR-1_Weekday |52088404-DEC20-D02CAR-1_Weekday |null         |1           |6110300 |6110029_DEC20|10611000290801|
|611-13139|DEC20-D02CAR-1_Weekday |52088405-DEC20-D02CAR-1_Wee

In [7]:
shapes = spark.read.csv('hdfs://namenode:8020/ola/static_data/shapes.txt', header=True)
shapes.show()

+-----------+-------------+---------------+-----------------+
|   shape_id| shape_pt_lat|   shape_pt_lon|shape_pt_sequence|
+-----------+-------------+---------------+-----------------+
|20999_DEC20|34.0455868426|-118.2530497433|            10001|
|20999_DEC20|34.0466699681| -118.252038928|            10002|
|20999_DEC20|34.0481153251|-118.2507294304|            10003|
|20999_DEC20|34.0482659327|-118.2505852918|            10004|
|20999_DEC20|34.0482659327|-118.2505852918|            20001|
|20999_DEC20|34.0495085066|-118.2493966237|            20002|
|20999_DEC20|34.0495250347|-118.2493768658|            20003|
|20999_DEC20|34.0509649284|-118.2480375502|            20004|
|20999_DEC20|34.0511799592|-118.2478318593|            20005|
|20999_DEC20|34.0511799592|-118.2478318593|            30001|
|20999_DEC20|34.0523938699|-118.2466717458|            30002|
|20999_DEC20|34.0536437629| -118.245513355|            30003|
|20999_DEC20|34.0540547946|-118.2451319326|            30004|
|20999_D

In [8]:
stops = spark.read.csv('hdfs://namenode:8020/ola/static_data/stops.txt', header=True)
stops.show(5)

+-------+---------+--------------------+---------+-----------+-----------+--------+-------------+--------------+---------+
|stop_id|stop_code|           stop_name|stop_desc|   stop_lat|   stop_lon|stop_url|location_type|parent_station|tpis_name|
+-------+---------+--------------------+---------+-----------+-----------+--------+-------------+--------------+---------+
|      1|        1| Paramount / Slauson|     null|  33.973248|-118.113113|    null|         null|          null|     null|
|      3|        3|    Jefferson / 10th|     null|  34.025471|-118.328402|    null|         null|          null|     null|
|      6|        6|120th / Augustus ...|     null|  33.924696|-118.242222|    null|         null|          null|     null|
|      7|        7|120th / Martin Lu...|     null|  33.924505|-118.240369|    null|         null|          null|     null|
|     12|       12|   15054 Sherman Way|     null|  34.201075|-118.461953|    null|         null|          null|     null|
+-------+-------

In [9]:
calendar = spark.read.csv('hdfs://namenode:8020/ola/static_data/calendar.txt', header=True)
calendar.show(5)

+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
|          service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|start_date|end_date|
+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
|DEC20-D01CAR-3_Su...|     0|      0|        0|       0|     0|       0|     1|  20201213|20210620|
|DEC20-D01CAR-1_We...|     1|      1|        1|       1|     1|       0|     0|  20201214|20210625|
|DEC20-D01CAR-2_Sa...|     0|      0|        0|       0|     0|       1|     0|  20201219|20210626|
|DEC20-D02CAR-3_Su...|     0|      0|        0|       0|     0|       0|     1|  20201213|20210620|
|DEC20-D02CAR-1_We...|     1|      1|        1|       1|     1|       0|     0|  20201214|20210625|
+--------------------+------+-------+---------+--------+------+--------+------+----------+--------+
only showing top 5 rows



In [10]:
calendar_dates = spark.read.csv('hdfs://namenode:8020/ola/static_data/calendar_dates.txt', header=True)
calendar_dates.show(5)

+--------------------+--------+--------------+
|          service_id|    date|exception_type|
+--------------------+--------+--------------+
|DEC20-D01CAR-3_Su...|20201225|             1|
|DEC20-D01CAR-3_Su...|20210101|             1|
|DEC20-D01CAR-3_Su...|20210531|             1|
|DEC20-D01CAR-1_We...|20201225|             2|
|DEC20-D01CAR-1_We...|20210101|             2|
+--------------------+--------+--------------+
only showing top 5 rows



In [11]:
stop_times = spark.read.csv('hdfs://namenode:8020/ola/static_data/stop_times.txt', header=True)
stop_times.show(5)

+--------------------+------------+--------------+-------+-------------+--------------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|       stop_headsign|pickup_type|drop_off_type|
+--------------------+------------+--------------+-------+-------------+--------------------+-----------+-------------+
|52088401-DEC20-D0...|    05:39:00|      05:39:00|  10246|            1|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:40:00|      05:40:00|  10248|            2|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:41:00|      05:41:00|   9371|            3|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:42:00|      05:42:00|   9350|            4|611 - Vernon Station|          0|            0|
|52088401-DEC20-D0...|    05:43:00|      05:43:00|   9351|            5|611 - Vernon Station|          0|            0|
+--------------------+------------+-----

In [12]:
route_shape_occurences = routes\
    .join(trips, "route_id")\
    .join(shapes, "shape_id")\
    .groupBy("route_id", "shape_id")\
    .count()

route_shape_occurences.show()

+---------+-------------+-----+
| route_id|     shape_id|count|
+---------+-------------+-----+
|163-13139|1630129_DEC20|24339|
| 16-13139| 160379_DEC20|46230|
|117-13139|1170068_DEC20|45108|
|251-13139|2510148_DEC20| 7866|
| 10-13139| 100567_DEC20|25200|
| 10-13139| 100613_DEC20|28495|
|200-13139|2000041_DEC20| 5592|
|734-13139|7340034_DEC20| 2116|
|217-13139|2170234_DEC20| 1866|
| 20-13139| 200723_DEC20|53312|
|794-13139|7940062_DEC20|  501|
| 51-13139| 510203_DEC20| 3660|
|260-13139|2600264_DEC20|  666|
|206-13139|2060176_DEC20|57441|
| 30-13139| 300791_DEC20| 1740|
| 51-13139| 510230_DEC20|16160|
| 66-13139| 660394_DEC20| 6468|
|234-13139|2340063_DEC20|16910|
|111-13139|1110319_DEC20| 7641|
|115-13139|1150493_DEC20| 2096|
+---------+-------------+-----+
only showing top 20 rows



In [13]:
routes_with_shape = route_shape_occurences\
    .withColumn('rn', row_number().over(Window.partitionBy('route_id').orderBy(col('count').desc())))\
    .where('rn = 1')\
    .select("route_id", "shape_id")\
    .join(shapes, "shape_id")\
    .select("route_id", col("shape_pt_lat").cast("double"), col("shape_pt_lon").cast("double"), col("shape_pt_sequence").cast("int"))

routes_with_shape.show()

+---------+-------------+---------------+-----------------+
| route_id| shape_pt_lat|   shape_pt_lon|shape_pt_sequence|
+---------+-------------+---------------+-----------------+
|102-13139| 33.950004218|-118.3928137447|            10001|
|102-13139|33.9507939301| -118.392821075|            10002|
|102-13139|33.9507951784|-118.3924376032|            10003|
|102-13139|33.9494320666|-118.3924058284|            10004|
|102-13139|33.9494294801| -118.392609587|            10005|
|102-13139|33.9494266345|-118.3928083984|            10006|
|102-13139|33.9488811994|-118.3928035327|            10007|
|102-13139| 33.948881809|-118.3928694803|            10008|
|102-13139|33.9488646177|-118.3939278131|            10009|
|102-13139|33.9488611374|-118.3941520089|            10010|
|102-13139|33.9488258448|-118.3948607487|            10011|
|102-13139|33.9487825858|-118.3954837223|            10012|
|102-13139|33.9487613347|-118.3958469771|            10013|
|102-13139|33.9487613347|-118.3958469771

In [42]:
routes_segments = routes_with_shape\
    .withColumn("id", monotonically_increasing_id())\
    .select("*", *[lead(col(c)).over(Window.orderBy("id")).alias("next_" + c) for c in ["shape_pt_lat", "shape_pt_lon", "shape_pt_sequence"]])\
    .drop("id")\
    .dropna("any")\
    .where(floor(col("shape_pt_sequence") / 10000) == floor(col("next_shape_pt_sequence") / 10000))\
    .select(
        split("route_id", "-").getItem(0).alias("route_id"), "shape_pt_lat", "shape_pt_lon", "next_shape_pt_lat", "next_shape_pt_lon", 
        col("shape_pt_sequence").alias("segment_sequence"), floor(col("shape_pt_sequence") / 20000 + 0.5).alias("segment_id"),
        haversine_meter("shape_pt_lat", "shape_pt_lon", "next_shape_pt_lat", "next_shape_pt_lon").alias("segment_len_meter")
    )

routes_segments.cache()
routes_segments.printSchema()
routes_segments.show(1000)
routes_segments.count()

root
 |-- route_id: string (nullable = true)
 |-- shape_pt_lat: double (nullable = true)
 |-- shape_pt_lon: double (nullable = true)
 |-- next_shape_pt_lat: double (nullable = true)
 |-- next_shape_pt_lon: double (nullable = true)
 |-- segment_sequence: integer (nullable = true)
 |-- segment_id: long (nullable = true)
 |-- segment_len_meter: double (nullable = true)

+--------+-------------+---------------+-----------------+-----------------+----------------+----------+-------------------+
|route_id| shape_pt_lat|   shape_pt_lon|next_shape_pt_lat|next_shape_pt_lon|segment_sequence|segment_id|  segment_len_meter|
+--------+-------------+---------------+-----------------+-----------------+----------------+----------+-------------------+
|     102| 33.950004218|-118.3928137447|    33.9507939301|   -118.392821075|           10001|         1|  87.81458205525286|
|     102|33.9507939301| -118.392821075|    33.9507951784|  -118.3924376032|           10002|         1|  35.37099699284967|
|    

62722

# Load and process historical data

In [21]:
# Load data with filename as request time
bus_positions = spark\
    .read\
    .json("hdfs://namenode:8020/ola/historical_data/20210512T0*/*")\
    .withColumn("filepath", input_file_name())\
    .select(substring("filepath", -18, 10).alias("req_time"), explode("items").alias("vehicle_position"))\
    .select(col("req_time").cast("long"), col("vehicle_position.*"))\

# calculate timestamp of record as: req_time - seconds_since_report
bus_positions = bus_positions\
    .withColumn("timestamp", col("req_time") - col("seconds_since_report"))\
    .select("id", "latitude", "longitude", "route_id", substring("run_id", -1, 1).alias("direction"), "timestamp")\

# Drop rows with same id, latitude, longitude, route_id, direction (keep row with lowest timestamp)
bus_positions = bus_positions\
    .orderBy(["id", "route_id", "timestamp", asc("req_time")])\
    .dropDuplicates(["id", "latitude", "longitude", "route_id", "direction"])\

# add day_type field, 0 if is weekday, 1 if is weekend or holiday
bus_positions = bus_positions\
    .withColumn("day_type", get_day_type("timestamp"))

# add timebin field (each timebin is 10 minutes, so there will be 144 timebins a day)
bus_positions = bus_positions\
    .withColumn("timebin", floor((col("timestamp") % 86400) / 600))

# drop rows with at least one null column
bus_positions = bus_positions.dropna("any")

bus_positions.cache()
bus_positions.printSchema()
bus_positions.show(1000)
bus_positions.count()

root
 |-- id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- route_id: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- day_type: integer (nullable = true)
 |-- timebin: long (nullable = true)

+-----+----------+------------+--------+---------+----------+--------+-------+
|   id|  latitude|   longitude|route_id|direction| timestamp|day_type|timebin|
+-----+----------+------------+--------+---------+----------+--------+-------+
| 1665|34.0501785|-118.2452467|      92|        0|1620800102|       0|     37|
| 1730|34.1326423|-118.1964153|     181|        1|1620795248|       0|     29|
| 1748| 34.059765| -118.401307|      28|        1|1620799202|       0|     36|
| 1751| 34.241389|-118.5233218|     239|        1|1620796809|       0|     32|
| 1769|  33.93082| -118.184052|     120|        1|1620795838|       0|     30|
| 1769| 33.930495|-118.2160507|     120|        1

35784

In [43]:
# get segment id of bus positions
df = bus_positions\
    .join(routes_segments, "route_id")\
    .withColumn("bus_segment_distance", haversine_meter("latitude", "longitude", "shape_pt_lat", "shape_pt_lon") + haversine_meter("latitude", "longitude", "next_shape_pt_lat", "next_shape_pt_lon") - col("segment_len_meter"))\
    .withColumn("row_number", row_number().over(Window.partitionBy("id", "route_id", "latitude", "longitude", "direction").orderBy(asc("bus_segment_distance"))))\
    .where("row_number = 1")\
    .select(col("id").alias("bus_id"), "route_id", "latitude", "longitude", "direction", "timestamp", "day_type", "timebin", "segment_sequence", "segment_id", "bus_segment_distance")\
    .orderBy("bus_id", "route_id", "timestamp")

df.show(1000)

+------+--------+----------+------------+---------+----------+--------+-------+----------------+----------+--------------------+
|bus_id|route_id|  latitude|   longitude|direction| timestamp|day_type|timebin|segment_sequence|segment_id|bus_segment_distance|
+------+--------+----------+------------+---------+----------+--------+-------+----------------+----------+--------------------+
|  1508|     205|  33.82962| -118.290314|        1|1620795062|       0|     29|          700002|        35|  1.0762598287785465|
|  1508|     205| 33.794449| -118.290871|        1|1620795433|       0|     29|          600006|        30|  0.9531388574654329|
|  1508|     205| 33.789719| -118.282417|        1|1620795597|       0|     29|          550004|        28|  1.2661250959109722|
|  1508|     205| 33.788177| -118.280907|        1|1620795702|       0|     30|          580008|        29|  2.0843897274228524|
|  1508|     205| 33.789413|  -118.29966|        1|1620795944|       0|     30|          520005| 